In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
import gin
import numpy as np
from matplotlib import pyplot as plt
from torch.autograd import Variable
from tqdm.auto import tqdm
import torch
from causal_util.helpers import lstdct2dctlst
from sparse_causal_model_learner_rl.sacred_gin_tune.sacred_wrapper import load_config_files
from sparse_causal_model_learner_rl.loss.losses import fit_loss_obs_space, lagrangian_granular
from sparse_causal_model_learner_rl.config import Config
from sparse_causal_model_learner_rl.learners.rl_learner import CausalModelLearnerRL
%matplotlib inline
gin.enter_interactive_mode()

In [9]:
def reload_config():
    load_config_files(['../sparse_causal_model_learner_rl/configs/rl_const_sparsity_obs_space.gin',
#                        '../keychest/config/5x5_1f_obs.gin',
#                        '../sparse_causal_model_learner_rl/configs/env_kc_5x5_1f_obs_quad.gin',
                       '../sparse_causal_model_learner_rl/configs/env_sm5_linear.gin',
                       '../sparse_causal_model_learner_rl/configs/with_lagrange_dual_sparsity.gin',
    #                    '../sparse_causal_model_learner_rl/configs/with_lagrange_dual_sparsity_per_component.gin',
                      ])

In [10]:
reload_config()
gin.bind_parameter('Config.collect_initial_steps', 1000)

In [11]:
l = CausalModelLearnerRL(Config())

2021-04-15 07:47:03,739	WARNING session.py:20 -- Session not detected. You should not be calling this function outside `tune.run` or while using the class API. 


Selecting name 0158df6a-9dae-11eb-bfe7-90e2baa15a70
Make environment SparseMatrix-v0 [<class 'encoder.observation_encoder.LinearMatrixEncoder'>] {}
(pid=28970) Make environment SparseMatrix-v0 [<class 'encoder.observation_encoder.LinearMatrixEncoder'>] {}
(pid=28968) Make environment SparseMatrix-v0 [<class 'encoder.observation_encoder.LinearMatrixEncoder'>] {}



(pid=28973) Make environment SparseMatrix-v0 [<class 'encoder.observation_encoder.LinearMatrixEncoder'>] {}


In [12]:
l.create_trainables()

[False, False, False, False]
[22, 256, 256, 256, 1] 6
[False]
[5, 1] 5
[False]
[6, 1] 5


In [13]:
ctx = l.collect_and_get_context()

In [14]:
from sparse_causal_model_learner_rl.loss.losses import cache_get, maybe_item, delta_pow2_sum1, delta_01_obs, manual_switch_gradient, RunOnce

In [15]:
l.model.model.switch.probas.data[:, :] = 0.5

In [16]:
l.lagrange_multipliers.vectorized

False

In [17]:
l.lagrange_multipliers().shape

torch.Size([14])

In [18]:
fit_loss_obs_space(**ctx, 
                   fill_switch_grad=True, divide_by_std=True, loss_local_cache={},
                  return_per_component=True)

{'loss': tensor(326.1825, device='cuda:0', grad_fn=<MeanBackward1>),
 'losses': {'additional': 0.0,
  'obs': tensor([98.0338, 56.7195, 42.0653, 56.6127, 57.4453], device='cuda:0',
         grad_fn=<MeanBackward1>),
  'feat': tensor([1.4787, 1.7663, 1.5031, 0.8975, 0.9573, 1.0633], device='cuda:0',
         grad_fn=<MeanBackward1>),
  'feat_model': tensor([1.3600, 1.7022, 1.4947, 0.7931, 1.2504, 1.0392], device='cuda:0',
         grad_fn=<MeanBackward1>),
  'additional_orig': 0.0,
  'obs_orig': tensor([50.5862, 55.0631, 55.6485, 48.6480, 52.5029], device='cuda:0',
         grad_fn=<MeanBackward1>),
  'feat_orig': tensor([1.0841, 1.0301, 1.0439, 1.0167, 1.0127, 1.0072], device='cuda:0',
         grad_fn=<MeanBackward1>),
  'feat_model_orig': tensor([1.0776, 1.0481, 1.0359, 1.0225, 1.0314, 1.0734], device='cuda:0',
         grad_fn=<MeanBackward1>)},
 'metrics': {'mean_feature': array([-9.2498714e-01, -1.8447432e-01,  3.1731811e-01,  2.6983058e-01,
          1.2752929e-01, -1.1920930e-10]

In [19]:
from sparse_causal_model_learner_rl.loss.losses import fit_loss_obs_space, lagrangian_granular
reload_config()

In [20]:
lagrangian_granular(**ctx, mode='PRIMAL')

sparsity [shape=torch.Size([])] -> min
fit/obs [shape=torch.Size([])] <= 0.008, lm_index=3 required=True
fit/feat [shape=torch.Size([])] <= 0.005, lm_index=1 required=True
fit/feat_model [shape=torch.Size([])] <= 0.005, lm_index=2 required=True
fit_all_on/obs [shape=torch.Size([])] <= 0.008, lm_index=11 required=True
fit_all_on/feat [shape=torch.Size([])] <= 0.005, lm_index=9 required=True
fit_all_on/feat_model [shape=torch.Size([])] <= 0.005, lm_index=10 required=True
fit_all_half/obs [shape=torch.Size([])] <= 0, lm_index=3 required=False
fit_all_half/feat [shape=torch.Size([])] <= 0, lm_index=1 required=False
fit_all_half/feat_model [shape=torch.Size([])] <= 0, lm_index=2 required=False


{'loss': tensor(734.1140, device='cuda:0', grad_fn=<AddBackward0>),
 'metrics': {'fit': {'value': 391.3555908203125,
   'coeff': 1.0,
   'mean_feature': array([-9.2498714e-01, -1.8447432e-01,  3.1731811e-01,  2.6983058e-01,
           1.2752929e-01, -1.1920930e-10], dtype=float32),
   'std_feature': array([4.7014923, 3.0951374, 3.8547962, 3.0942852, 2.8349895, 0.5782171],
         dtype=float32),
   'min_feature': -16.050722122192383,
   'max_feature': 15.205986976623535,
   'loss_fcons': 6.185311794281006,
   'loss_add': 0.0,
   'loss_rec': 262.43780517578125,
   'loss_fcons_pre': 6.289677143096924,
   'rec_fit_acc_loss_01_agg': 1.480400025844574,
   'loss_discrete': 116.44278717041016,
   'loss_orig': 274.91280364990234},
  'fit/additional': {'value': 0.0},
  'fit/obs': {'value': 391.3555908203125},
  'fit/feat': {'value': 8.569881439208984},
  'fit/feat_model': {'value': 8.903656959533691},
  'fit/additional_orig': {'value': 0.0, 'coeff': 1.0},
  'fit/obs_orig': {'value': 262.437805

In [21]:
lagrangian_granular(**ctx, mode='DUAL')

sparsity [shape=torch.Size([])] -> min
fit/obs [shape=torch.Size([])] <= 0.008, lm_index=3 required=True
fit/feat [shape=torch.Size([])] <= 0.005, lm_index=1 required=True
fit/feat_model [shape=torch.Size([])] <= 0.005, lm_index=2 required=True
fit_all_on/obs [shape=torch.Size([])] <= 0.008, lm_index=11 required=True
fit_all_on/feat [shape=torch.Size([])] <= 0.005, lm_index=9 required=True
fit_all_on/feat_model [shape=torch.Size([])] <= 0.005, lm_index=10 required=True
fit_all_half/obs [shape=torch.Size([])] <= 0, lm_index=3 required=False
fit_all_half/feat [shape=torch.Size([])] <= 0, lm_index=1 required=False
fit_all_half/feat_model [shape=torch.Size([])] <= 0, lm_index=2 required=False


{'loss': tensor(-589.3839, device='cuda:0', grad_fn=<NegBackward>),
 'metrics': {'fit': {'value': 203.06185913085938,
   'coeff': 1.0,
   'mean_feature': array([-9.2498714e-01, -1.8447432e-01,  3.1731811e-01,  2.6983058e-01,
           1.2752929e-01, -1.1920930e-10], dtype=float32),
   'std_feature': array([4.7014923, 3.0951374, 3.8547962, 3.0942852, 2.8349895, 0.5782171],
         dtype=float32),
   'min_feature': -16.050722122192383,
   'max_feature': 15.205986976623535,
   'loss_fcons': 6.187043190002441,
   'loss_add': 0.0,
   'loss_rec': 262.5189208984375,
   'loss_fcons_pre': 6.287208080291748,
   'rec_fit_acc_loss_01_agg': 1.476599931716919,
   'loss_discrete': -71.93131256103516,
   'loss_orig': 274.99317169189453},
  'fit/additional': {'value': 0.0},
  'fit/obs': {'value': 203.06185913085938},
  'fit/feat': {'value': 6.483997821807861},
  'fit/feat_model': {'value': 9.288799285888672},
  'fit/additional_orig': {'value': 0.0, 'coeff': 1.0},
  'fit/obs_orig': {'value': 262.51892